# Job Generation for CAVs
Author Ilias Parmaksizoglou

This notebook provided a quick tour on how to use the Job Genenation for CAVs tool (2.3). THe needed input is :
* O-D matrix of target destinations and travel times inside the port
* Schedule with expected ETA (Estimated Time of Arrival) and approximate time needed inside the port by each truck

The output of this tool is needed by further tools and contains feasible bundles of trucks. A schematic representation of the output of this tool is presented below: 

![Alt](core/bundle.png)

In [1]:
import gurobipy as gp
import pandas as pd
from itertools import combinations,permutations
import os

# Loading the network
The two inputs are stored in the core folder, in the files "time_matrix.xlsx" and "truck_arrivals.xslx"

In [2]:
# Loading Data
core_dir = os.path.join(os.getcwd(),"core")
network_namefile = "time_matrix.xlsx"
schedule_namefile = "truck_arrivals.csv"

network = pd.read_excel(os.path.join(core_dir,network_namefile))
schedule = pd.read_csv(os.path.join(core_dir,schedule_namefile))

no_destinations = len(set(schedule['Destination']))
no_windows = len(set(schedule['Window']))

# Creating arcs and cost of traversal for network
multidict_input = {}
for i,row in network.iterrows():
    multidict_input[row[0],row[1]] = row[2]
arcs,time_between = gp.multidict(multidict_input)


# Main Loop
Algorithm Explanation for later

In [8]:
# Empty DataFrame to store generated jobs
jobs = pd.DataFrame()

# Outer loop for job generation at each time-window
for w in range(1,no_windows + 1):
    
    # Isolate trucks for specific window w
    win_trucks = schedule.loc[schedule["Window"] == w]
    
    # Inner loop for job generation for bundles of length up to d
    for d in range(1,no_destinations+1):
        truck_comb = list(combinations(win_trucks["Truck"],d))

        # Storing performance of examined combinations
        comb_performance = {}
        
        for comb in truck_comb:
        
            # Loading Input specific to combination 
            trucks = list(comb)

            # Isolating job's information for specific bundle
            trucks_input = {}
            for i,row in win_trucks.iterrows():
                if row[0] in trucks:
                    trucks_input[row[0]] = row[2],row[3],row[4],row[5]
            trucks,target_dest,arrival,time_needed,chaperon= gp.multidict(trucks_input)
            destinations = list(set(target_dest.values()))

            # Exit Loop if trucks in the combination are going to the same dest - Illegal Move
            if len(destinations) != d:
                continue

            # Exit Loop if a truck in the combination does not need chaperoning 
            ind = False
            for t in trucks:
                if chaperon[t] == 0 and len(trucks)!=1:
                    ind= True
                    continue

            # Possible States based on number of destinations
            states = list(range(1,len(destinations)+1))
            destinations.append("Gate")
            
            # Possible Combinations of ordering 
            indices = list(permutations(comb, len(states)))
            indices = list(set(indices))

            # Initialize earliest job start
            idle_gate_time = 0
            earliest_start = max(arrival.values())
            for t in trucks: 
                idle_gate_time += earliest_start-arrival[t]

            # Determined Traversal inside the port
            min_cost = 1e+4 
            for index in indices:
                travel_cost = []
                finish_time = []
                evac_time = []
                entry_time = []
                start = "Gate"
                dur = 0
                for t in index:
                    travel_cost.append(2*time_between[start,target_dest[t]])
                    finish_time.append(sum(travel_cost) + time_needed[t])
                    evac_time.append(sum(travel_cost)/2 + time_needed[t])
                    entry_time.append(earliest_start+dur+time_between[start,target_dest[t]])
                    dur +=time_between[start,target_dest[t]]
                    start = target_dest[t]      

                for a,t in enumerate(reversed(index)):
                    if a == 0:
                        start = target_dest[t]
                        evac_time[-1-a]= earliest_start+ evac_time[-1-a]
                    else:
                        if a == len(index):
                            break
                        time_back = time_between[start,target_dest[t]]
                        evac_time[-1-a]= max(earliest_start+evac_time[-1-a],evac_time[-a]+time_back)
                        start = target_dest[t]
                
                # Travel Cost Equal to extra time and travel cost
                turnaround = max(finish_time)
                cost = turnaround + idle_gate_time
                if cost < min_cost:
                    best_ind = index
                    min_cost = cost
                    best_turnaround =  turnaround
                    destinations = [target_dest[x] for x in best_ind]
                    evac_time = [x for x in evac_time]
                    entry_time = [x for x in entry_time]
                    comb_performance[best_ind] = [min_cost,best_turnaround,destinations,evac_time,entry_time]

            # Store results in dictionary
            results = {}
            results["W"] = [w]
            for c in comb_performance.items():
                results["T"] = [c[1][0]]
                results["Sq"] = [",".join(c[0])]
                results["ET"] = [max(arrival.values())]
                results["LT"] = (sum(results["ET"]) + sum([c[1][1]]))
                results["En"] =[",".join(f"{x-results['ET'][0]}" for x in c[1][4])]    
                results["Ex"] =[",".join(f"{x-results['ET'][0]}" for x in c[1][3])]    
                results["F"] =[",".join(str(x) for x in c[1][2])]
                    
            jobs = jobs.append(pd.DataFrame.from_dict(results))

jobs = jobs.set_index("Sq")
jobs.to_csv(os.path.join(core_dir,"jobs.csv"))